In [25]:
import xarray as xr
from ndpyramid.utils import set_zarr_encoding
import numpy as np

In [26]:
# url = "s3://carbonplan-oae-efficiency/v2/store1b_rechunked.zarr/"
# current = xr.open_dataset(url, engine='zarr', chunks={})
# current

In [27]:
path = '/global/homes/a/abanihi/OAE_efficiency_corrected_.nc'
ds = xr.open_dataset(path)
ds = set_zarr_encoding(ds, float_dtype=np.float32, int_dtype=np.int32)
ds

<xarray.Dataset> Size: 4MB
Dimensions:         (region: 4, season: 4, polygon: 300, time: 189)
Coordinates:
  * region          (region) <U14 224B 'Pacific' 'Atlantic' ... 'Southern_Ocean'
  * season          (season) <U7 112B 'January' 'April' 'July' 'October'
  * polygon         (polygon) int32 1kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
  * time            (time) object 2kB 0347-01-16 12:00:00 ... 0362-09-16 00:0...
Data variables:
    OAE_efficiency  (region, season, polygon, time) float32 4MB 0.03391 ... nan

In [28]:
# Initialize an empty list to store valid datasets
datasets = []

# Iterate over each combination of polygon, region, and season
for polygon in ds.polygon:
    for region in ds.region:
        for season in ds.season:
            # Select the data for the current combination
            data = ds.sel(polygon=polygon, region=region, season=season)
            
            # Drop NaN values based on OAE_efficiency
            cond = data.OAE_efficiency.notnull()
            results = data.where(cond, drop=True)
            
            # Ensure time is sorted and remove duplicates
            if "time" in results.dims:
                results = results.sortby('time')
                #results = results.isel(time=results.get_index("time").drop_duplicates(keep="first"))

            # Check if the time dimension has 180 values
            if results.sizes.get('time', 0) == 180:
                datasets.append(results)

# Combine all valid datasets into a single dataset
try:
    dset = xr.combine_by_coords(datasets)
    print("Combined dataset successfully")
except ValueError as e:
    print("Error combining datasets:", e)

Error combining datasets: Resulting object does not have monotonic global indexes along dimension time


In [13]:
atlantic = ds.sel(region='Atlantic').isel(polygon=slice(0, 150))
pacific = ds.sel(region='Pacific').isel(polygon=slice(0, 200))
south_atlantic = ds.sel(region='South').isel(polygon=slice(0, 300))
southern = ds.sel(region='Southern_Ocean').isel(polygon=slice(0, 40))

In [14]:
atlantic

<xarray.Dataset> Size: 456kB
Dimensions:         (season: 4, polygon: 150, time: 189)
Coordinates:
    region          <U14 56B 'Atlantic'
  * season          (season) <U7 112B 'January' 'April' 'July' 'October'
  * polygon         (polygon) int32 600B 0 1 2 3 4 5 ... 144 145 146 147 148 149
  * time            (time) object 2kB 0347-01-16 12:00:00 ... 0362-09-16 00:0...
Data variables:
    OAE_efficiency  (season, polygon, time) float32 454kB 0.01765 ... 0.7921

In [20]:
dset = xr.concat([atlantic, pacific, south_atlantic, southern], dim='polygon').drop_vars('region')
dset = dset.rename_vars({'polygon': 'polygon_id', 'season': 'injection_date'}).swap_dims({'polygon': 'polygon_id', 'season': 'injection_date'})#.reset_index()
dset['polygon_id'] = np.arange(0, 690, dtype=np.int32)
dset['injection_date'] = np.array([1, 4, 7, 10], dtype=np.int32)
dset

<xarray.Dataset> Size: 2MB
Dimensions:         (injection_date: 4, polygon_id: 690, time: 189)
Coordinates:
  * injection_date  (injection_date) int32 16B 1 4 7 10
  * polygon_id      (polygon_id) int32 3kB 0 1 2 3 4 5 ... 685 686 687 688 689
  * time            (time) object 2kB 0347-01-16 12:00:00 ... 0362-09-16 00:0...
Data variables:
    OAE_efficiency  (injection_date, polygon_id, time) float32 2MB 0.01765 .....

In [24]:
dset.isel(polygon_id=0)

<xarray.Dataset> Size: 5kB
Dimensions:         (injection_date: 4, time: 189)
Coordinates:
  * injection_date  (injection_date) int32 16B 1 4 7 10
    polygon_id      int32 4B 0
  * time            (time) object 2kB 0347-01-16 12:00:00 ... 0362-09-16 00:0...
Data variables:
    OAE_efficiency  (injection_date, time) float32 3kB 0.01765 ... 0.4494